In [1]:
#include <iostream>

/*a workaround to solve cling issue*/
#include "../inc/macos_cling_workaround.hpp"
/*set libtorch path, load libs*/
#include "../inc/load_libtorch.hpp"
/*import custom defined macros*/
#include "../inc/custom_def.hpp"
/*import matplotlibcpp*/
#include "../inc/load_matplotlibcpp.hpp"
/*import opencv*/
#include "../inc/load_opencv.hpp"

/*import libtorch header file*/
#include <torch/torch.h>
#include <opencv2/opencv.hpp>
#include <cmath>

// Use (void) to silent unused warnings.
#define assertm(exp, msg) assert(((void)msg, exp))

In [2]:
#define VAR_NAME(Variable) (#Variable)

void print_tensor_size(std::string name, torch::Tensor t)
{
    int dims = t.dim();
    std::cout << name << " dims is (";
    for (int i = 0; i < dims; i++) {
        std::cout << t.size(i);
        if (i < (dims - 1)) std::cout << " x ";
    }
    std::cout << ")" << std::endl;
}

# 多输入多输出通道

**实现一下多输入通道互相关运算**

In [3]:
// def corr2d_multi_in(X, K):
//     return sum(d2l.corr2d(x, k) for x, k in zip(X, K))

In [4]:
torch::Tensor corr2d(torch::Tensor X, torch::Tensor K)
{
    int h = K.size(0);
    int w = K.size(1);
    
    torch::Tensor Y = torch::zeros({X.size(0) - h + 1, X.size(1) - w + 1});
    for (int i = 0; i < Y.size(0); i++) {
        for (int j = 0; j < Y.size(1); j++) {
            Y[i][j] = (X.index({torch::indexing::Slice(i, i+h), torch::indexing::Slice(j, j+w)}) * K).sum();
//             printT(Y[i][j]);
        }        
    }
    
    return Y;
}

In [5]:
torch::Tensor corr2d_multi_in(torch::Tensor X, torch::Tensor K)
{
    assert(X.dim() == K.dim());
    int chan = X.size(0);
    torch::Tensor Y;
    for (int i = 0; i < chan; i++) {
        if (Y.numel() == 0) {
            Y = corr2d(X[i], K[i]);
        } else {
            Y += corr2d(X[i], K[i]);
        }
    }
    return Y;
}

**验证互相关运算的输出**

In [6]:
// X = torch.tensor([[[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]],
//                   [[1.0, 2.0, 3.0], [4.0, 5.0, 6.0], [7.0, 8.0, 9.0]]])
// K = torch.tensor([[[0.0, 1.0], [2.0, 3.0]], [[1.0, 2.0], [3.0, 4.0]]])

// corr2d_multi_in(X, K)

In [7]:
auto X = torch::tensor(at::ArrayRef<float>({0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 
                                       1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0}));
X = X.reshape({2, 3, 3});

auto K = torch::tensor(at::ArrayRef<float>({0.0, 1.0, 2.0, 3.0, 1.0, 2.0, 3.0, 4.0}));

K = K.reshape({2, 2, 2});

auto Z = corr2d_multi_in(X, K);

printT(Z);

Z = 
  56   72
 104  120
[ CPUFloatType{2,2} ]
<<--->>



**计算多个通道的输出的互相关函数**

In [8]:
// def corr2d_multi_in_out(X, K):
//     return torch.stack([corr2d_multi_in(X, k) for k in K], 0)

// K = torch.stack((K, K + 1, K + 2), 0)
// K.shape

In [9]:
torch::Tensor corr2d_multi_in_out(torch::Tensor X, torch::Tensor K)
{
    int chan = K.size(0);
    std::vector<torch::Tensor> Y;
    for (int i = 0; i < chan; i++) {
        Y.push_back(corr2d_multi_in(X, K[i]));
    }
    
    return torch::stack(at::TensorList(Y));
}

In [10]:
K = torch::stack({K, K+1, K+2}, 0);
printT(K.sizes());

K.sizes() = 
[3, 2, 2, 2]
<<--->>



In [11]:
printT(corr2d_multi_in_out(X, K));

corr2d_multi_in_out(X, K) = 
(1,.,.) = 
   56   72
  104  120

(2,.,.) = 
   76  100
  148  172

(3,.,.) = 
   96  128
  192  224
[ CPUFloatType{3,2,2} ]
<<--->>



**1x1卷积**

In [12]:
// def corr2d_multi_in_out_1x1(X, K):
//     c_i, h, w = X.shape
//     c_o = K.shape[0]
//     X = X.reshape((c_i, h * w))
//     K = K.reshape((c_o, c_i))
//     Y = torch.matmul(K, X)
//     return Y.reshape((c_o, h, w))

// X = torch.normal(0, 1, (3, 3, 3))
// K = torch.normal(0, 1, (2, 3, 1, 1))

// Y1 = corr2d_multi_in_out_1x1(X, K)
// Y2 = corr2d_multi_in_out(X, K)
// assert float(torch.abs(Y1 - Y2).sum()) < 1e-6

In [13]:
torch::Tensor corr2d_multi_in_out_1x1(torch::Tensor X, torch::Tensor K)
{
    assert(X.dim() == K.dim());
    int c_i = X.size(0);
    int h   = X.size(1);
    int w   = X.size(2);
    int c_o = K.size(0);
    X = X.reshape({c_i, h*w});
    K = K.reshape({c_o, c_i});
    auto Y = torch::matmul(K, X);
    return Y.reshape({c_o, h, w});
}

In [14]:
auto X1 = torch::normal(0, 1, {3, 3, 3});
auto K1 = torch::normal(0, 1, {2, 3, 1, 1});
auto Y1 = corr2d_multi_in_out_1x1(X1, K1);
auto Y2 = corr2d_multi_in_out(X1, K1);
printT(Y1);
printT(Y2);

Y1 = 
(1,.,.) = 
  0.0736  1.2673 -0.6003
 -1.1960 -1.8473  1.6022
 -0.7439  0.8201 -0.2694

(2,.,.) = 
  0.6711 -0.8463  1.3777
 -0.1810 -0.9293 -1.3548
 -0.2085  0.6384 -0.7410
[ CPUFloatType{2,3,3} ]
<<--->>

Y2 = 
(1,.,.) = 
  0.0736  1.2673 -0.6003
 -1.1960 -1.8473  1.6022
 -0.7439  0.8201 -0.2694

(2,.,.) = 
  0.6711 -0.8463  1.3777
 -0.1810 -0.9293 -1.3548
 -0.2085  0.6384 -0.7410
[ CPUFloatType{2,3,3} ]
<<--->>



In [15]:
printT((torch::abs(Y1 - Y2).sum()));

(torch::abs(Y1 - Y2).sum()) = 
1.56462e-07
[ CPUFloatType{} ]
<<--->>

